# Demo of Predictive State Smoothing (PRESS)

PRESS is a kernel smoothing technique for any type of predictive learning problem (classification, regression, ...).

This notebook shows examples of how to use the main functionality for a regression and classification problem.  Core PRESS functionality and layers, though, can be used for any learning problem with non-standard activation functions.

See also


* Goerg (2018) *[Classification using Predictive State Smoothing (PRESS): A scalable kernel classifier for high-dimensional features with variable selection](https://research.google/pubs/pub46767/)*.

* Goerg (2017) *[Predictive State Smoothing (PRESS): Scalable non-parametric regression for high-dimensional data with variable selection](https://research.google/pubs/pub46141/).*


In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [2]:
import importlib
import os
import sys
import pathlib
import tensorflow as tf

from os.path import dirname
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import logging
import datetime
parent_path = dirname(pathlib.Path(pathlib.Path(os.getcwd())))

if parent_path not in sys.path:
    sys.path.insert(0, parent_path)

In [3]:
from pypress.keras import layers
from pypress.keras import initializers
from pypress.keras import regularizers
from pypress import utils

importlib.reload(layers)

<module 'pypress.keras.layers' from '/home/georg/Projects/pypress/pypress/keras/layers.py'>

# Utility functions and setup

In [ ]:
# misc helper functions
from typing import Tuple, Any
import sklearn
import sklearn.model_selection
from sklearn.preprocessing import RobustScaler


def _get_loss_activation_metrics(y: pd.Series) -> Tuple[Any, Any, Any]:

    if len(np.unique(y)) == 2:
        act = "sigmoid"
        loss_fn = "binary_crossentropy"
        metrics = [tf.keras.metrics.AUC(curve="PR", name="aupr"), tf.keras.metrics.AUC(curve="ROC", name="auc_roc")]
    else:
        act = "linear"
        loss_fn = "mse"
        metrics = [tf.keras.metrics.mean_squared_error]
        
        if (y >= 0.).all():
            act = "softplus"
            loss_fn = "mse"

    return (loss_fn, act, metrics)


def _get_recommended_callbacks():
    logdir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    print(logdir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)
    return [tf.keras.callbacks.EarlyStopping(patience=10),
                             tensorboard_callback], logdir


def _scale_df(X, scaler = None):
    
    if scaler is None:
        scaler = RobustScaler()
        scaler.fit(X)
    return pd.DataFrame(scaler.transform(X), columns=X.columns, index=X.index), scaler


def _train_test_scale(X, y):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)
    
    X_train_s, scaler = _scale_df(X_train, None)
    X_test_s, scaler = _scale_df(X_test, scaler)
    
    return (X_train, y_train, X_test, y_test), scaler


# Regression


In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(as_frame=True)

In [ ]:
X, y = housing["data"], housing["target"]
X.shape, y.shape

In [ ]:
sns.clustermap(X.corr("spearman"))
plt.show()
sns.displot(y)

In [ ]:
X = X.loc[y < 5]
y = y.loc[y < 5]

X.shape, y.shape

In [ ]:
sns.clustermap(X.corr("spearman"))
plt.show()
sns.displot(y)

In [ ]:
scaled_data, sc = _train_test_scale(X, y)
X_tr, y_tr, X_ts, y_ts = scaled_data
X_tr.shape

In [ ]:
loss_fn, act, metrics = _get_loss_activation_metrics(y_ts)
print(act, loss_fn)

feat_input = tf.keras.layers.Input(shape=(X_tr.shape[1],))

feat_eng_layer = tf.keras.Sequential()
feat_eng_layer.add(tf.keras.layers.Dense(30, "relu", kernel_regularizer=tf.keras.regularizers.L2(0.001)))
feat_eng_layer.add(tf.keras.layers.Dropout(0.25))
feat_eng_layer.add(tf.keras.layers.BatchNormalization())
feat_eng_layer.add(tf.keras.layers.Dense(10, "tanh", kernel_regularizer=tf.keras.regularizers.L2(0.001)))
feat_eng_layer.add(tf.keras.layers.Dropout(0.25))
feat_eng_layer.add(tf.keras.layers.BatchNormalization())

hidden = feat_eng_layer(feat_input)
eps_mapping = layers.PredictiveStateSimplex(n_states=5, activity_regularizer=regularizers.Uniform(l1=0.1),
                                            kernel_regularizer=tf.keras.regularizers.l2(0.001))
pred_states = eps_mapping(hidden)
state_mean_layer = layers.PredictiveStateMeans(units=1, activation=act)
pred = state_mean_layer(pred_states)

mod = tf.keras.Model(inputs=feat_input, outputs=pred)
mod.compile(loss=loss_fn, optimizer=tf.keras.optimizers.Nadam(learning_rate=0.005),
            metrics=metrics)
mod.summary()

In [ ]:
eps_mapping.trainable_weights

In [ ]:
state_mean_layer.trainable_variables

In [ ]:
tf.keras.utils.plot_model(mod)

In [ ]:
history = mod.fit(X_tr, y_tr, epochs=10, batch_size=32,
                  validation_data=(X_ts, y_ts),
                  callbacks=[],#_get_recommended_callbacks()
                 )

In [ ]:
mod.layers[-1].state_conditional_means.numpy().ravel()

In [ ]:
sns.heatmap(eps_mapping.trainable_weights[0].numpy())

In [ ]:
sns.displot(y)

In [ ]:
y_pred = mod.predict(X)

sns.jointplot(y, y_pred.ravel(), kind="hex")

In [ ]:
state_mapping = tf.keras.Model(inputs=feat_input, outputs=pred_states)
print(X_ts.shape)
pred_emb = state_mapping(X_ts.values).numpy()
pred_emb = pd.DataFrame(pred_emb, index=X_ts.index)
sns.heatmap(pred_emb)

In [ ]:
sns.pairplot(pred_emb)

In [ ]:
pred_emb.shape

In [ ]:
utils.col_normalize(pred_emb).sum(axis=0)

In [ ]:
X_ts.head()

In [ ]:
utils.state_size(pred_emb)

In [ ]:
utils.agg_data_by_state(pred_emb, X_ts).round(2)

# Classification

In [ ]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True, as_frame=True)

In [ ]:
sns.clustermap(X.corr("spearman"))

In [ ]:
y.value_counts(normalize=True).plot.bar()
plt.grid()

In [ ]:
scaled_data, sc = _train_test_scale(X, y)
X_tr, y_tr, X_ts, y_ts = scaled_data
X_tr.shape, X_ts.shape

In [ ]:
loss_fn, act, metrics = _get_loss_activation_metrics(y)

feat_input = tf.keras.layers.Input(shape=(X.shape[1],))

feat_eng_layer = tf.keras.Sequential()
feat_eng_layer.add(tf.keras.layers.Dense(50, "selu"))
feat_eng_layer.add(tf.keras.layers.Dropout(0.25))
feat_eng_layer.add(tf.keras.layers.BatchNormalization())
feat_eng_layer.add(tf.keras.layers.Dense(30, "tanh"))
feat_eng_layer.add(tf.keras.layers.Dropout(0.25))
feat_eng_layer.add(tf.keras.layers.BatchNormalization())

hidden = feat_eng_layer(feat_input)
pred_states = layers.PredictiveStateSimplex(n_states=5, activity_regularizer=regularizers.Uniform(10.))(hidden)
pred = layers.PredictiveStateMeans(units=1, activation=act)(pred_states)

mod = tf.keras.Model(inputs=feat_input, outputs=pred)
mod.compile(loss=loss_fn, optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
            metrics=metrics)
mod.summary()

In [ ]:
clbks, logdir_str = _get_recommended_callbacks()

In [ ]:
history = mod.fit(X_tr, y_tr, epochs=30, 
                  validation_data=(X_ts, y_ts),
                  callbacks=clbks,
                 )

In [ ]:
%tensorboard --logdir $logdir_str

In [ ]:
preds = pd.DataFrame({"true": y_ts, "pred": mod.predict(X_ts).ravel()})

In [ ]:
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay)
precision, recall, _ = precision_recall_curve(preds["true"], preds["pred"])
disp = PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()


In [ ]:
state_mapping = tf.keras.Model(inputs=feat_input, outputs=pred_states)
print(X_ts.shape)
pred_emb = pd.DataFrame(state_mapping(X_ts.values).numpy(), index=X_ts.index)
sns.heatmap(pred_emb)

In [ ]:
sns.pairplot(pred_emb)

In [ ]:
X_state = utils.agg_data_by_state(X_ts, pred_emb)
X_state

In [ ]:
mod.layers[-1].state_conditional_means.numpy()

In [ ]:
utils.state_size(pred_emb, normalize=True)

In [ ]:
X_state

In [ ]:
sns.heatmap(pred_emb.dot(utils.col_normalize(pred_emb).transpose()))

## Explain predictions

In [ ]:
pred_fn = lambda x: mod.predict(x)[:, 0]

In [ ]:
import shap
# build a Permutation explainer and explain the model predictions on the given dataset
explainer = shap.explainers.Permutation(pred_fn, X_ts)
explainer.feature_names

In [ ]:
shap_values = explainer(X_ts.iloc[:20])  # ~20sec

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.plots.waterfall(shap_values[5])

In [ ]:
shap.plots.waterfall(shap_values[0])

# Example of single `PRESS()` layer

In [ ]:
mod = tf.keras.Sequential()
mod.add(layers.PredictiveStateSimplex(n_states=6, activity_regularizer=regularizers.Uniform(0.01), input_dim=X.shape[1]))
mod.add(layers.PredictiveStateMeans(units=1, activation="sigmoid"))
mod.compile(loss=loss_fn, optimizer=tf.keras.optimizers.Nadam(learning_rate=0.01),
            metrics=metrics)
mod.summary()

In [ ]:
mod.fit(X_tr, y_tr, epochs=2)

In [ ]:
mod = tf.keras.Sequential()
mod.add(layers.PRESS(units=1, n_states=6, activation="sigmoid", activity_regularizer=regularizers.DegreesOfFreedom(l1=0.1, df=2.), input_dim=X.shape[1]))
mod.compile(loss=loss_fn, optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
            metrics=metrics)
mod.summary()

In [ ]:
mod.fit(X_tr, y_tr, epochs=12)

In [10]:
from sklearn.datasets import load_breast_cancer
import sklearn
X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_s = sklearn.preprocessing.robust_scale(X)  # See demo.ipynb to properly scale X with train/test split


import tensorflow as tf
from pypress.keras import layers
from pypress.keras import regularizers

mod = tf.keras.Sequential()
# see layers.PRESS() for single layer wrapper
mod.add(layers.PredictiveStateSimplex(
            n_states=6,
            activity_regularizer=regularizers.Uniform(0.01),
            input_dim=X.shape[1]))
mod.add(layers.PredictiveStateMeans(units=1, activation="sigmoid"))
mod.compile(loss="binary_crossentropy",
            optimizer=tf.keras.optimizers.Nadam(learning_rate=0.01),
            metrics=[tf.keras.metrics.AUC(curve="PR", name="auc_pr")])
mod.summary()
mod.fit(X_s, y, epochs=10, validation_split=0.2)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 predictive_state_simplex_6   (None, 6)                186       
 (PredictiveStateSimplex)                                        
                                                                 
 predictive_state_means_6 (P  (None, 1)                6         
 redictiveStateMeans)                                            
                                                                 
Total params: 192
Trainable params: 192
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
15/15 [==============================] - 1s 17ms/step - loss: 0.5911 - auc_pr: 0.9795 - val_loss: 0.4856 - val_auc_pr: 0.9975
Epoch 2/10
15/15 [==============================] - 0s 3ms/step - loss: 0.4933 - auc_pr: 0.9935 - val_loss: 0.4265 - val_auc_pr: 0.9995
Epoch 3/10
15/15 [=============================